# Автоматизация сбора и визуализации данных агрегатора новостей

Цель - анализ взаимодействия пользователей с карточками Яндекс.Дзен. Надо наладить ежедневное получение данных о взаимодействии, а также построение на их основе дашборда для менеджеров по анализу контента.


**Использованные данные**: 

Таблица dash_visits. 
- record_id — первичный ключ,
- item_topic — тема карточки,
- source_topic — тема источника,
- age_segment — возрастной сегмент,
- dt — дата и время,
- visits — количество событий.

**Состав данных для дашборда**:

- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:

- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

Характер данных:

- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta


In [2]:
!pip install psycopg2-binary


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [70]:
# pd.io.sql.read_sql(query, con = engine) 

In [4]:
query = '''
            SELECT *
            FROM dash_visits
        '''

In [5]:
dash_visits = pd.io.sql.read_sql(query, con = engine)
print(dash_visits.head(5))

   record_id item_topic source_topic age_segment                  dt  visits
0    1040597     Деньги         Авто       18-25 2019-09-24 18:32:00       3
1    1040598     Деньги         Авто       18-25 2019-09-24 18:35:00       1
2    1040599     Деньги         Авто       18-25 2019-09-24 18:54:00       4
3    1040600     Деньги         Авто       18-25 2019-09-24 18:55:00      17
4    1040601     Деньги         Авто       18-25 2019-09-24 18:56:00      27


In [6]:
dash_visits.to_csv('dash_visits.csv', index=False)

In [8]:
print(dash_visits['dt'].min(), dash_visits['dt'].max())

2019-09-24 18:28:00 2019-09-24 19:00:00


In [9]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [10]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [11]:
visits_groupby = dash_visits.groupby('item_topic').agg({'visits': 'sum'})

In [12]:
visits_groupby.sort_values(by='visits', ascending=False)

,visits
item_topic,
Наука,21736
Отношения,20666
Интересные факты,19942
Общество,19640
Подборки,17772
Россия,16966
Полезные советы,15435
История,15389
Семья,11897


In [13]:
visits_groupby_source = dash_visits.groupby('source_topic').agg({'item_topic': 'nunique'})

In [14]:
visits_groupby_source.sort_values(by='item_topic', ascending=False)

,item_topic
source_topic,
Авто,25
Одежда,25
Спорт,25
Семья,25
Семейные отношения,25
Россия,25
Ремонт,25
Путешествия,25
Психология,25


## Итоги:

1. [Дашборд](https://public.tableau.com/views/__16656871653700/sheet4?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link "Дашборд"), который был построен в Tableau Public
2. [Презентация](https://disk.yandex.ru/i/Mr-Y98iqDX9H0g "Презентация") дашборда